In [ ]:
!pip install  git+https://github.com/himanshudabas/twint.git@origin/master#egg=twint
#gitインストール要

!pip install nlplot
!pip install janome
!conda install MeCab #pipだとうまくいかなかった・・・

In [ ]:
import twint
import nest_asyncio

nest_asyncio.apply()

# Configure
c = twint.Config()
#特定のユーザー名
c.Username = "twitter"
#実行日から数えて過去何日分のツイートを取得するか
c.Limit = 30
#csv形式で保存するか
c.Store_csv = True
#保存ファイルの名前
#c.Output = "Sample.csv"

# Run
twint.run.Search(c)

In [ ]:
import twint
import nest_asyncio

#これないとエラーが出る・・・・
nest_asyncio.apply()

def Twitter_Scraper(user_name,since_date,until_date,output_filename):
    c = twint.Config()
    c.Username = user_name #解析するユーザー名
    c.Store_json = True # Store_csvもあるがカラム名が入らなかったため不採用
    c.Output = output_filename #上記アウトプットするJSONファイルの名前
    c.Since = since_date #ここから
    c.Until = until_date #ここまで
    c.Pandas = True #出力形式をPandasにする
    c.Hide_output = True # False にするとツイートがターミナルに出力
    twint.run.Search(c)

output_file = "twint_scrape.json"
Twitter_Scraper("Qiita", "2020-09-18 00:00:00", "2021-09-18 13:00:00", output_file)

df = twint.storage.panda.Tweets_df #データフレーム形式へ変換
Tweets_df = df['tweet'].to_frame() #tweetカラムのみデータフレーム形式で抽出
tweets_list = Tweets_df['tweet'].tolist() #tweetカラムをlist化して抽出しておく

In [1]:
import re
import MeCab

def word_extraction(text):
    """文を取得して特定名詞だけ抽出する関数"""

    #邪魔なURL部分を正規表現で排除
    result = re.sub("https?://[\w!\?/\+\-_~=;\.,\*&@#\$%\(\)'\[\]]+", "", text)
    parse = tagger.parse(result) #解析

    lines = parse.split('\n') #改行で区切る
    nounlist = [] #特定名詞を格納するリスト
    for line in lines:
        feature = line.split('\t') #タブで区切る
        if len(feature) == 2: #'EOS'と''を省く
            info = feature[1].split(',') #カンマで区切る
            hinshi = info[0] #その0番目(品詞)を取得
            hinshi_classi = info[1] #1番目(品詞分類)を取得
            #名詞 - 一般,サ変,固有名詞　だけ抽出する　※抽出したいものに合わせてカスタマイズすればいい
            if (hinshi in ('名詞')) and ((hinshi_classi in ('一般')) or (hinshi_classi in ('サ変接続')) or (hinshi_classi in ('固有名詞'))):
                nounlist.append(feature[0]) #info[6]を取得してもいいが、今回は原文を取得
    return nounlist

#wordsカラムを↑の関数で挿入する
Tweets_df['words'] = Tweets_df['tweet'].apply(word_extraction)

Tweets_df.head(1) #1行表示する


ModuleNotFoundError: No module named 'MeCab'